In [1]:
import matplotlib.pyplot as plt
from IPython.display import clear_output
from multiTracker import CentroidTracker

In [2]:
import numpy as np
import imutils
import cv2
cap = cv2.VideoCapture('depthData/depth_vids1/14.avi')
import time

In [3]:
#Thresholds

KERNEL_MORPH_DILATE_SIZE=25
KERNEL_DILATE_ITER=5
CONTOUR_AREA_MIN=70*70
CONTOUR_AREA_MAX=180*180
#DEPTH_IMAGE_MAX=5000
DEPTH_MASK_THRESH=150
THRESH_RETAIN_PEAKS = 235
KERNEL_MORPH_OPEN_SIZE=9

TRACKER_RETAIN_FRAMES=10
TRACKER_MAX_DIST_THESH=80
TRACKER_LINE_BOUNDARY=300
THRESH_RATIO_AREA=0.55
THRESH_WH_RATIO=0.75

ROI_STARTX=10
ROI_STARTY=350

ROI_ENDX=420
ROI_ENDY=920

In [4]:

kernel_size=KERNEL_MORPH_DILATE_SIZE
kernel_size_open= KERNEL_MORPH_OPEN_SIZE
kernel = np.ones((kernel_size,kernel_size),np.uint8)
kernel2 = np.ones((kernel_size_open,kernel_size_open),np.uint8)

def findHead(image,kernel_size=25,thresh_rem=THRESH_RETAIN_PEAKS, show= 0):
    
    #Get the binary mask
    image[image>230]=0
    displayx(image,"orig",show)
    thresh = cv2.threshold(image, DEPTH_MASK_THRESH, 255,cv2.THRESH_BINARY)[1]
    displayx(thresh,"mask",show)
    thresh[thresh==255]=1
    
    #Apply dilate op-> replace all the values with local maximaum
    dilated = cv2.dilate(image,kernel,iterations=5)
    displayx(dilated,"dilated",show)
    
    #Aplly mask on dilated image
    res= dilated*thresh
    displayx(res,"final dilated",show)
    
    #Subtract the dilated image from original->the local maxima valuea will be ~= 0
    res= res-image
    displayx(res,"",show)
    
    #replace the unmasked values with max so that we get boundary of maxima
    res[thresh==0]=255
    displayx(res,"",show)
   
    #Inverse the image-> maxima become highlighted
    res= 255-res
    displayx(res,"",show)
    
    #Apply threshold to extract just the maxima(top values)
    res[res< thresh_rem] =0
    displayx(res,"",show)
    
    #Apply closing operation to remove holes from the max values
    res= cv2.morphologyEx(res, cv2.MORPH_OPEN, kernel2)
    
    displayx(res,"final",show)
    return res

In [5]:
def displayx(img,title,show=0):
    if show==0 :
        return
    plt.figure(figsize=(10, 8))
    plt.imshow(img)
    plt.title(title) 
    plt.show()

In [6]:
def overlap(circle,circles):
    ((x,y),r)=circle
    for circle1 in circles:
        ((x1,y1),r1)=circle1
        # check if distance between centres is less than r1 + r2
        if (x1 - x)*(x1 - x) + (y1 - y)*(y1 - y) < (r1+r)*(r1+r):
            return True
    return False

In [7]:
def preprocess(cnts):
    circles=[]
    for cnt in cnts:
        #get the area
        area = cv2.contourArea(cnt)
        # apply the threshold to get contours with reasonable area
        if area > CONTOUR_AREA_MIN  and area < CONTOUR_AREA_MAX:
            x,y,w,h = cv2.boundingRect(cnt)
            wh_ratio=w/h
            #Check width to height ratio is in reasonable range
            if(wh_ratio> THRESH_WH_RATIO and wh_ratio < 1/THRESH_WH_RATIO):
                circle = cv2.minEnclosingCircle(cnt)
                ((cx,cy),r)=circle
                circle_area = 22/7*r*r
                #Check if the actual area and enclosing circle area is in reasonable range
                ratio= area/circle_area
                if ratio > THRESH_RATIO_AREA and r>60:
                    circles.append(circle)
                    #Check if there exists and overlapping circle already
                    #if overlap(circle,circles): continue
    return circles

In [10]:
import os
k=0
duration=0
#Instantiate the tracker
tracker = CentroidTracker(TRACKER_RETAIN_FRAMES,TRACKER_MAX_DIST_THESH,TRACKER_LINE_BOUNDARY,10)
for l in range (2,22):
    cap = cv2.VideoCapture('depthData/depth_vids1/'+str(l)+'.avi')
    #tracker = CentroidTracker(4,50,250)
    direc = 'frames_vid_13'
    #os.mkdir(direc)
    
    while(cap.isOpened()):
        start = time.time()
        ret, frame = cap.read()
        if ret<=0:
            break
        k+=1
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray =gray[ROI_STARTX:ROI_ENDX,ROI_STARTY:ROI_ENDY]
        final = findHead(255-gray)

        #Get all Contours
        cnts = cv2.findContours(final.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        
        #Preprocess the contours
        circles= preprocess(cnts)

#       # Display output
        objects,radius,boxes = tracker.update(circles,gray)
        duration += time.time()-start
        if(k%100==0):
            print('FPS: ',k/duration,k,duration)
#         if boxes is not None:
#             for i, newbox in enumerate(boxes):
#                 p1 = (int(newbox[0]), int(newbox[1]))
#                 p2 = (int(newbox[0] + newbox[2]), int(newbox[1] + newbox[3]))
#                 #cv2.rectangle(gray, p1, p2, (255, 255, 255), 2, 1)
#         #print (objects)
#         # loop over the tracked objects
#         for (objectID, centroid) in objects.items():
#             # draw both the ID of the object and the centroid of the
#             # object on the output frame
#             text = "ID {}".format(objectID)
#             cv2.putText(gray, text, (int(centroid[0]) - 10, int(centroid[1]) - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
#             cv2.circle(gray, (int(centroid[0]),int(centroid[1])), radius[objectID], (255, 255, 255), 2)
        
#         #Draw counter info
#         counter_text = 'IN: '+str(tracker.personIn)+ "  OUT: " +str(tracker.personOut)
#         cv2.putText(gray, counter_text, (20,20),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
#         cv2.putText(gray, str(k), (20,100),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
#         cv2.line(gray, (0,300), (600,300), (255, 255, 255), 2)
        
#         #Plot Image
#         displayx(gray,"output",1)
#         clear_output(wait=True)

FPS:  177.4379128654315 100 0.56357741355896
FPS:  104.37868450083903 200 1.916100025177002
FPS:  70.0018636881777 300 4.285600185394287
FPS:  58.86849245651228 400 6.794806241989136
FPS:  66.00708223228521 500 7.574944734573364
FPS:  59.382300359361466 600 10.104020833969116
FPS:  65.56573581741827 700 10.676308155059814
FPS:  67.72960720462251 800 11.811673402786255
FPS:  50.56702130271733 900 17.798161268234253
FPS:  54.28513235015338 1000 18.421250104904175
FPS:  54.459269941021844 1100 20.19858145713806
FPS:  46.50160127312466 1200 25.80556297302246
FPS:  45.59804379988404 1300 28.509994983673096
FPS:  47.18499917631518 1400 29.670446634292603
FPS:  48.58112509668901 1500 30.87618899345398
FPS:  47.994389503212616 1600 33.33722996711731
FPS:  49.15850819339747 1700 34.5820095539093


KeyboardInterrupt: 

In [9]:
overlap(x,y,r,circles)

NameError: name 'x' is not defined

In [ ]:
for circle in circles:
    #if (circle.x - x)*(circle.x - x) + (circle.y - y)*(circle.y - y) < (circle.r+r)*(circle.r+r):
    print(circle)

In [ ]:
cap = cv2.VideoCapture('depthData/depth_vids1/14.avi')
ret, frame = cap.read()

In [ ]:
frame = cv2.imread(direc+"/127.jpg")

In [ ]:
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
gray = gray[100:350,350:920]


In [ ]:
out = findHead(255-gray,show=1)

In [ ]:
cnts = cv2.findContours(out.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)

In [ ]:
for cnt in cnts:
    area = cv2.contourArea(cnt)
    ((x,y),r)= cv2.minEnclosingCircle(cnt)
    area_circle = 22/7*r*r
    cv2.circle(out, (int(x),int(y)), int(r), (255, 255, 255), 2)
    displayx(out,"",1)
    x,y,w,h = cv2.boundingRect(cnt)
    
    print("r",r)
    print("area",area)
    print("w/h",w/h)
    print("circle_area",area_circle)
    print("ratio",area/area_circle)


In [ ]:
plt.figure(figsize=(10, 8))

cv2.circle(frame, (1000, 100), 100, (255, 255, 255), 2)
plt.imshow(frame)

In [ ]:
plt.hist(gray.ravel(),bins='auto')

In [ ]:
img1 = np.copy(255-gray)
img2= np.copy(255-gray)
img1[img1<160]=0
img1[img1>235]=0
plt.imshow(img1)

In [ ]:
plt.hist(img1.ravel(),bins='auto')

In [ ]:
a=[[0,5,9],[2,1,3],[9,6,1]]

In [ ]:
a

In [ ]:
D=np.random.rand(3,3)

In [ ]:
D

In [ ]:
rows=D.min(axis=1).argsort()

In [ ]:
D.argmin(axis=1)[rows]

In [ ]:
rows